<a href="https://colab.research.google.com/github/priyanka0178885547/ML-LAB/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/titanic_train.csv')
df.shape

(891, 12)

In [3]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df=df.drop(['PassengerId','Name','Ticket'],axis=1)

In [5]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [6]:
[col for col in df.columns if df[col].isnull().sum()>0]


['Age', 'Cabin', 'Embarked']

In [7]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [8]:
df=df.drop(['Embarked','Cabin'],axis=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [10]:
from sklearn.metrics import roc_auc_score

In [11]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [12]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [13]:
X = pd.get_dummies(X,drop_first=True)


In [14]:
from sklearn.model_selection import train_test_split


In [15]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [16]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [17]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)


In [18]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    5.4s finished

[2025-08-19 16:03:37] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.0s finished

[2025-08-19 16:03:42] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.6s finished

[2025-08-19 16:03:45] Features: 3/5 -- score: 0.8409244272706652[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.7s finished

[2025-08-19 16:03:48] Features: 4/5 -- score: 0.8467381970772171[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s finished

[2025-08-19 16:03:50] Features: 5/5 -- score: 0.8448899906030265

In [19]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

best accracy score 0.8448899906030265
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


In [20]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)
sfs2=sfs2.fit(np.array(x_train.fillna(0)),y_train)
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

best accracy score 0.8449124704319367
best subset(indices) (0, 1, 3, 4, 5)
Best subset (corresponding names): ('0', '1', '3', '4', '5')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    6.1s finished

[2025-08-19 16:04:47] Features: 5/5 -- score: 0.8449124704319367

In [21]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 56/56

best accuracy score 0.8106193548387097
best subset(indices) (0, 1, 5)
Best subset (corresponding names): ('Pclass', 'Age', 'Sex_male')


In [22]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
11,"(1, 2)","[0.64, 0.568, 0.608, 0.5483870967741935, 0.604...",0.593845,"(Age, SibSp)",0.041393,0.032205,0.016103
34,"(1, 3, 4)","[0.584, 0.6, 0.592, 0.5887096774193549, 0.6290...",0.598748,"(Age, Parch, Fare)",0.020585,0.016016,0.008008
1,"(1,)","[0.632, 0.608, 0.608, 0.5887096774193549, 0.59...",0.606697,"(Age,)",0.018764,0.014599,0.007299
3,"(3,)","[0.568, 0.6, 0.624, 0.6048387096774194, 0.6451...",0.6084,"(Parch,)",0.03307,0.02573,0.012865
12,"(1, 3)","[0.56, 0.568, 0.64, 0.6290322580645161, 0.6451...",0.608439,"(Age, Parch)",0.047226,0.036743,0.018372
13,"(1, 4)","[0.592, 0.616, 0.624, 0.5887096774193549, 0.63...",0.611561,"(Age, Fare)",0.023917,0.018608,0.009304
31,"(1, 2, 3)","[0.64, 0.584, 0.68, 0.5645161290322581, 0.6290...",0.61951,"(Age, SibSp, Parch)",0.052853,0.041121,0.020561
2,"(2,)","[0.624, 0.624, 0.656, 0.5645161290322581, 0.71...",0.637252,"(SibSp,)",0.064192,0.049944,0.024972
51,"(1, 2, 3, 4)","[0.656, 0.648, 0.656, 0.6129032258064516, 0.62...",0.638774,"(Age, SibSp, Parch, Fare)",0.023454,0.018248,0.009124
21,"(0, 1, 2)","[0.616, 0.616, 0.712, 0.5967741935483871, 0.65...",0.6388,"(Pclass, Age, SibSp)",0.052594,0.04092,0.02046


In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0, criterion='gini')
rfc.fit(x_train, y_train)
for feature in zip(x_train.columns, rfc.feature_importances_):
    print(feature)


('Pclass', np.float64(0.08580548090109545))
('Age', np.float64(0.26433757901109173))
('SibSp', np.float64(0.05355992352286061))
('Parch', np.float64(0.03699908167792097))
('Fare', np.float64(0.28790097031129963))
('Sex_male', np.float64(0.2713969645757317))


In [24]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfc, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- Age
- Fare
- Sex_male


In [25]:
from sklearn.metrics import accuracy_score
rfc_full = RandomForestClassifier(random_state=0, criterion='gini')
rfc_full.fit(x_train, y_train)
pred_full = rfc_full.predict(x_test)
print('The accuracy of classifier with full features: {:.2f}'.format(accuracy_score(y_test, pred_full)))


The accuracy of classifier with full features: 0.82


In [26]:
x_important_train = sfm.transform(x_train)
x_important_test = sfm.transform(x_test)

In [27]:
rfc_lim = RandomForestClassifier(random_state=0, criterion='gini')
rfc_lim.fit(x_important_train, y_train)
pred_lim = rfc_lim.predict(x_important_test)
print('The accuracy of classifier with limited features: {:.2f}'.format(accuracy_score(y_test, pred_lim)))

The accuracy of classifier with limited features: 0.80
